In [1]:
import gensim
import pandas as pd
import numpy as np
import nltk
from os import path

/usr/local/lib/python3.5/dist-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
filename = 'bank_train'
df = pd.read_csv(filename+'.csv')
df_test = pd.read_csv('bank_test_etalon.csv')

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
#we can add stop words

import pymorphy2
def tokenize(text):
    text = text.lower()
    text_list = nltk.word_tokenize(text)
    morph = pymorphy2.MorphAnalyzer()
    text_list = [word for word in text_list if word[0] != '@' and len(word) > 1 and word.isalpha()]
    text_list = [morph.parse(word)[0].normal_form for word in text_list]
    return text_list

In [4]:
subdir = '/home/parkin/Data/'
df_list = ['reviews1.csv', 'reviews2.csv', 'spr.csv', 'hospitals.csv', 'kinopoisk_bottom100.csv', 'kinopoisk_top250.csv']
text_columns = ['text', 'text', 'text', 'content', 'content', 'content']

In [5]:
import pickle

In [ ]:
all_sentences = {}
for df_name, text_column in zip(df_list[-2:], text_columns[-2:]):
    df = pd.read_csv(path.join(subdir, df_name))
    print (df_name)
    print ('~~~~~~~~`')
    df_len = len(df[text_column])
    result = []
    for idx, row in enumerate(df[text_column]):
        result.append(tokenize(row))
        if idx % 500 == 0:
            print ('{0}/{1} {2:.2f}%'.format(idx+1,df_len, 100.0 * idx / df_len))
    with open(df_name+'.pkl', 'wb') as fout:
        pickle.dump(result, fout, pickle.HIGHEST_PROTOCOL)
    all_sentences[df_name] = result

In [43]:
all_sentences = {}
for file in df_list:
    with open(file+'.pkl', 'rb') as fin:
        l = pickle.load(fin)
    all_sentences[file] = l

In [30]:
#don't use
for v in all_sentences.values():
    for idx, row in enumerate(v):
        v[idx] = ' '.join(row)
    print ('~~~~~~')

~~~~~~
~~~~~~
~~~~~~
~~~~~~
~~~~~~
~~~~~~


In [31]:
#WARNING
with open('all_sentences.pkl', 'wb') as fout:
    pickle.dump(all_sentences, fout, protocol=pickle.HIGHEST_PROTOCOL)

In [44]:
first_sentences = all_sentences['reviews1.csv']

In [46]:
model = gensim.models.Word2Vec(sentences=first_sentences,size=200)

In [6]:
model = gensim.models.Word2Vec.load('word2vec_normalize.model')

In [27]:
model.most_similar('средне')

[('посредственно', 0.6553890705108643),
 ('средненький', 0.599341630935669),
 ('суховато', 0.5942553877830505),
 ('неплохо', 0.5631283521652222),
 ('стандартно', 0.5505111217498779),
 ('сносно', 0.5501429438591003),
 ('съедобный', 0.5392283201217651),
 ('бледно', 0.5392034649848938),
 ('сносный', 0.537426233291626),
 ('недурно', 0.5361967086791992)]

In [49]:
for df_name in df_list[1:]:
    sentence = all_sentences[df_name]
    print (df_name)
    model.train(sentence)
    print ('2' + df_name)

reviews2.csv
2reviews2.csv
spr.csv
2spr.csv
hospitals.csv
2hospitals.csv
kinopoisk_bottom100.csv
2kinopoisk_bottom100.csv
kinopoisk_top250.csv
2kinopoisk_top250.csv


In [77]:
model.save('word2vec_normalize.model')
model.save_word2vec_format('word2vec_normalize.bin', binary=True)

In [16]:
v = list(model.vocab.keys())
companies = ['alfabank', 'gazprom', 'raiffeisen', 'rshb', 'sberbank', 'uralsib', 'vtb']

In [17]:
def filter_df(_df, _2vec, corporation):
    x_arr = []
    y_arr = []
    for i in range(len(_df)):
        label = _df[corporation][i]
        if label == '0' or label == '-1' or label == '1' or label == 0.0 or label == -1.0 or label == 1.0:
            x_arr.append(_2vec(_df['text'][i]))
            y_arr.append(int(label))
    X = np.array(x_arr)
    y = np.array(y_arr)
    print (len(X))
    print (len(y))
    return X, y

In [32]:
def sentence2vec(sentence, model):
    token_list = tokenize(sentence)
    vocab = list(model.vocab.keys())
    vector_size = model.vector_size
    token_list = [model[x] if x in vocab else np.zeros(vector_size) for x in token_list ]
    if len(token_list) > 0:
        return 1.0/len(token_list) * sum(token_list)
    else:
        return np.zeros(vector_size)
sen_to_vec = lambda x: sentence2vec(x, model) 

In [33]:
X, y = filter_df(df_test, sen_to_vec, companies[0])

536
536


In [34]:
for companie in companies[1:]:
    X_cur, y_cur = filter_df(df_test, sen_to_vec, companie)
    X = np.concatenate((X,X_cur), axis=0)
    y = np.concatenate((y,y_cur), axis=0)

137
137
150
150
99
99
2833
2833
60
60
651
651


In [35]:
X.shape #train data word2vec

(4466, 200)

In [42]:
X_test = X
y_test = y

In [29]:
X_train = X
y_train = 

In [44]:
import time
from sklearn.svm import SVC
from sklearn.metrics import f1_score

In [45]:
print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print ('SVM: ')
start_time = time.time()
model = SVC()
model.fit(X_train, y_train)
t = int(time.time() - start_time)
print ('fit time = {0}m{1}sec'.format(t // 60, t % 60))
start_time = time.time()
y_predict = model.predict(X_test)
t = int(time.time() - start_time)
print ('predict time = {0}m{1}sec'.format(t // 60, t % 60))
print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
SVM: 
fit time = 0m6sec
predict time = 0m4sec
f-macro: 0.4334506801174829
f-micro: 0.771383788625168


In [46]:
import my_score
func= lambda x,y: my_score.f_macro(my_score.cound_diff(x,y))
print ('strange f score - {0}'.format(func(y_predict, y_test)))

strange f score - 0.4546572057744066


In [47]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predict)

array([[ 275,  389,    0],
       [ 299, 3169,    0],
       [  49,  284,    1]])